In [67]:
MAP_HEIGHT = 80
MAP_WIDTH = 80
N_SPLITS = 5

In [177]:
from collections import defaultdict
from typing import Any, Union
import numpy as np
import numpy.random as rnd

class SubDungeon:
    
    SPLIT_MIN = 0.4
    SPLIT_MAX = 0.6
    
    # vertical split == vertical splitting line
    
    def __init__(self, extents : Union[tuple, None], parent : Any = None):
        
        self.left = None
        self.right = None
        self.parent = None
        self.extents = extents
        xmin, xmax, ymin, ymax = extents
        self.vertical_split = True
        if xmax - xmin < ymax - ymin:
            self.vertical_split = False
        if parent:
            self.parent = parent
        
    def split(self):
        if not self.extents:
            return []
        xmin, xmax, ymin, ymax = self.extents
        left_extents = None
        right_extents = None
        if self.vertical_split:
            minrange = xmin + int((xmax - xmin) * self.SPLIT_MIN)
            maxrange = xmin + int((xmax - xmin) * self.SPLIT_MAX)
            xrange = list(range(minrange, maxrange))
            if len(xrange) >= 3:
                split_x = rnd.choice(xrange)
                left_extents = (xmin, split_x, ymin, ymax)
                right_extents = (split_x, xmax, ymin, ymax)
        else:
            minrange = ymin + int((ymax - ymin) * self.SPLIT_MIN)
            maxrange = ymin + int((ymax - ymin) * self.SPLIT_MAX)
            yrange = list(range(minrange, maxrange))
            if len(yrange) >= 3:
                split_y = rnd.choice(yrange)
                left_extents = (xmin, xmax, ymin, split_y)
                right_extents = (xmin, xmax, split_y, ymax)   
        if left_extents and right_extents:
            self.left = SubDungeon(left_extents, self)
            self.right = SubDungeon(right_extents, self)
            return [self.left, self.right]
        return []
    
    def get_connector(self):
        connector = (None, None)
        if not self.left or not self.right:
            return connector
        xmin, xmax, ymin, ymax = self.right.extents
        if self.vertical_split:
            minrange = ymin + 1
            maxrange = ymax - 1
            yrange = list(range(minrange, maxrange))
            
            # I don't want to have connectors on walls
            if not self.right.vertical_split and self.right.right:
                yrange.remove(self.right.right.extents[2])
            if not self.left.vertical_split and self.left.right:
                try:
                    yrange.remove(self.left.right.extents[2])
                except ValueError:
                    # It is possible that both children' walls will be at the same level
                    pass
            
            conn_y = rnd.choice(yrange)
            connector = (xmin, conn_y)
        else:
            minrange = xmin + 1
            maxrange = xmax - 1
            xrange = list(range(minrange, maxrange))
            
            # I don't want to have connectors on walls
            if self.right.vertical_split and self.right.right:
                xrange.remove(self.right.right.extents[0])
            if self.left.vertical_split and self.left.right:
                try:
                    xrange.remove(self.left.right.extents[0])
                except ValueError:
                    # It is possible that both children' walls will be at the same level
                    pass
            
            conn_x = rnd.choice(xrange)
            connector = (conn_x, ymin)
        return connector
    
    def get_center(self):
        xmin, xmax, ymin, ymax = self.extents
        xcenter = xmin + int((xmax - xmin)/2)
        ycenter = ymin + int((ymax - ymin)/2)
        return (xcenter, ycenter)
    
    def get_corners(self):
        xmin, xmax, ymin, ymax = self.extents
        tl = (xmin + 1, ymin + 1)
        tr = (xmax, ymin + 1)
        bl = (xmin + 1, ymax)
        br = (xmax, ymax)
        return (tl, tr, bl, br)
               
    def print_tree(self):
        if self.left:
            self.left.print_tree()
        print(str(self))
        if self.right:
            self.right.print_tree()
        
    def __repr__(self):
        return (f"Vsplit: {self.vertical_split}, extents: {self.extents}, "
                f"parent: {self.parent.extents if self.parent else 'None'}")

class IndoorMap:
    
    N_SPLITS = 5
    
    def __init__(self, width : int, height : int, seed : Any):
        self.MAP_WIDTH = width
        self.MAP_HEIGHT = height
        self.spawn_points = []
        self.treasure_locations = []
        self.entrance = None
        self.exit = None
        self.hint_location = None
        self.seed = seed
        
        rnd.seed(self.seed)
        self.grid = np.zeros((MAP_HEIGHT, MAP_WIDTH), dtype=int)
        self.root = SubDungeon(extents = (0, self.MAP_WIDTH - 1, 0, self.MAP_HEIGHT - 1))
        self.tree_layers = defaultdict(list)
        self.tree_layers[0].append(self.root)
        
    def create_map(self):
        self.__split_rooms()
        self.__add_rooms_to_grid()
        self.__add_border()
        self.__add_spawn_points()
        self.__add_treasure_locations()
        
    def __split_rooms(self):
        # Perform  `N_SPLITS` splits starting from root node
        for n in range(self.N_SPLITS):
            for sub_dungeon in self.tree_layers[n]:
                self.tree_layers[n+1].extend(sub_dungeon.split())
        
    def __add_rooms_to_grid(self):
        
        # Draw walls of leaf subdungeons
        for sub_dungeon in self.tree_layers[self.N_SPLITS]:
            xmin, xmax, ymin, ymax = sub_dungeon.extents
            for x in range(xmin, xmax):
                self.grid[x][ymin] = 1
                self.grid[x][ymax] = 1
            for y in range(ymin, ymax):
                self.grid[xmin][y] = 1
                self.grid[xmax][y] = 1
        
        # Make connectors between rooms
        for n in sorted(list(range(self.N_SPLITS)), reverse = True):
            for sub_dungeon in self.tree_layers[n]:
                x, y = sub_dungeon.get_connector()
                if x != None and y != None:
                    self.grid[x][y] = 0
    
    def __add_spawn_points(self):
        
        self.spawn_points = [leaf.get_center() for leaf in self.tree_layers[self.N_SPLITS] if rnd.choice([True, False], p=[0.7, 0.3])]
        self.spawn_points = sorted(self.spawn_points, key = lambda x: (x[0], x[1]))
        self.entrance = self.spawn_points.pop(0)
        self.exit = self.spawn_points.pop()
    
    def __add_treasure_locations(self):
        
        for leaf in self.tree_layers[self.N_SPLITS]:
            # The treasure doesn't have to be in every single room
            if rnd.choice([True, False], p=[0.7, 0.3]):
                # Random corner of the room
                corner_no = rnd.choice(4)
                self.treasure_locations.append(leaf.get_corners()[corner_no])
        self.hint_location = self.treasure_locations.pop(rnd.choice(len(self.treasure_locations)))
    
    def __add_border(self):
        for y, row in enumerate(self.grid):
            for x, _ in enumerate(row):
                # I'd like the boundary of the map to stay as walls :D
                if x == 0 or x == self.MAP_WIDTH-1 or y == 0 or y == self.MAP_HEIGHT -1:
                    self.grid[y][x] = 99
        
    def get_map_dict(self) -> dict:
        return { 
            "grid" : self.grid.tolist(),
            "spawn_points" : self.spawn_points,
            "treasure_locations" : self.treasure_locations,
            "hint_location" : self.hint_location,
            "map_type" : "indoor",
            "entrance" : self.entrance,
            "exit" : self.exit,                 
        }     

In [175]:
grid = np.zeros((MAP_HEIGHT, MAP_WIDTH), dtype=int)

dungeon_tree_root = SubDungeon(extents=(0, MAP_WIDTH - 1, 0, MAP_HEIGHT - 1))
dungeon_tree = defaultdict(list)
dungeon_tree[-1].append(dungeon_tree_root)
for n in range(N_SPLITS):
    print(f"---------------LEVEL {n}---------------")
    for sub_dungeon in dungeon_tree[n-1]:
        sub_dungeon.print_tree()
        dungeon_tree[n].extend(sub_dungeon.split())
        
for sub_dungeon in dungeon_tree[N_SPLITS - 1]:
    
    xmin, xmax, ymin, ymax = sub_dungeon.extents
    for x in range(xmin, xmax):
        grid[x][ymin] = 1
        grid[x][ymax] = 1
    for y in range(ymin, ymax):
        grid[xmin][y] = 1
        grid[xmax][y] = 1

for n in sorted(list(range(N_SPLITS-1)), reverse = True):
    for sub_dungeon in dungeon_tree[n]:
        x, y = sub_dungeon.get_connector()
        if x != None and y != None:
            grid[x][y] = 0
        
print(grid.tolist())

---------------LEVEL 0---------------
Vsplit: True, extents: (0, 79, 0, 79), parent: None
---------------LEVEL 1---------------
Vsplit: False, extents: (0, 42, 0, 79), parent: (0, 79, 0, 79)
Vsplit: False, extents: (42, 79, 0, 79), parent: (0, 79, 0, 79)
---------------LEVEL 2---------------
Vsplit: True, extents: (0, 42, 0, 39), parent: (0, 42, 0, 79)
Vsplit: True, extents: (0, 42, 39, 79), parent: (0, 42, 0, 79)
Vsplit: False, extents: (42, 79, 0, 40), parent: (42, 79, 0, 79)
Vsplit: False, extents: (42, 79, 40, 79), parent: (42, 79, 0, 79)
---------------LEVEL 3---------------
Vsplit: False, extents: (0, 17, 0, 39), parent: (0, 42, 0, 39)
Vsplit: False, extents: (17, 42, 0, 39), parent: (0, 42, 0, 39)
Vsplit: False, extents: (0, 16, 39, 79), parent: (0, 42, 39, 79)
Vsplit: False, extents: (16, 42, 39, 79), parent: (0, 42, 39, 79)
Vsplit: True, extents: (42, 79, 0, 19), parent: (42, 79, 0, 40)
Vsplit: True, extents: (42, 79, 19, 40), parent: (42, 79, 0, 40)
Vsplit: True, extents: (42

In [176]:
mapgen = IndoorMap(MAP_WIDTH, MAP_HEIGHT, 123)
mapgen.create_map()
print(mapgen.get_map_dict())

{'grid': [[99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99, 99], [99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99], [99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 99], [99, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1